In [3]:
#Импортирование модуля Pandas, присвоение ему краткого имени pd.
import pandas as pd

In [4]:
#Импортирование исходного датасета, присвоение названия каждой колонке.
df = pd.read_csv('ds160216.csv', header = None, sep=',')
df.columns = ['uuid', 'short', 'long', 'label',  'date', 'source']

#### Подсчет количества уникальных групп лейблов новостей, исключая новости с лейблами "-", "S" и "Standard". 

In [ ]:
#Способ первый
num = 0
label_num = dict()
#Храним в словаре все лейблы, кроме "-", "S" и "Standard "
for label in df.label:
    if label != '-' and label != 'S' and label != 'Standard ':
        label_num[label] = 0
#Считаем количество записей в словаре
for label in label_num:
    num += 1
num

In [ ]:
#Способ второй
pd.Series.nunique(df['label']) - 3

#### Подсчет количества количества уникальных групп новостей, которые содержат более одной новости

In [ ]:
num = 0
label_num = dict()
#Храним в словаре все лейблы, кроме "-", "S" и "Standard " 
for label in df.label:
    if label != '-' and label != 'S' and label != 'Standard ':
        label_num[label] = 0
#Считаем, сколько раз встречается каждый лейбл в исходном датасете
for label in df.label:
    if label != '-' and label != 'S' and label != 'Standard ':
        label_num[label] += 1
#Считаем, сколько записей в словаре удовлетворяют условию, что количество новостей в группе больше одного
for label in label_num:
    if label_num[label] > 1:
        num += 1
num
#Эта процедура необходима для того, чтобы узнать, что существует необходимое для обучения классификатора количество
#новостей-дубликатов.

#### Построение гистограммы количества новостей в группе (группа с лейблами "-" не включена в гистограмму)

In [ ]:
import matplotlib
news_num_hist = pd.Series.value_counts(df.label)[1:]
print(news_num_hist.plot.hist(title = u"Гистограмма количества новостей в группе", logy = True))
#Гистограмма показывает количество случаев того, сколько лейбл содержит новостей. 
#Так, по гистограмме можно судить, что в датасете содержится огромное число новостей, которые не имеют дубликатов
#(что на порядок выше количества лейблов с двумя новостями).

#### Используя формулу для числа размещений, найдем полное количество пар новостей:
#### $C_n^2 = \binom{n}{2} = \frac{n!}{2!(n-2)!} = \frac{n(n-1)}{2}$

In [ ]:
((df.shape[0]) * (df.shape[0] - 1))/2
#Результат: 10293613903
#Видно, что полное число пар новостей очень велико, проводить обучение классификатора является невозможной задачей
#для одного компьютера. В связи с этим попробуем найти количество пар новостей, принадлежащих одной группе, а затем 
#создать датасет, содержащий равное количество пар новостей, принадлежащих одной группе, и пар новостей, не принадлежащих одной 
#одной группе.

### Создание попарного датасета

#### Создание датасета, содержащего пары новостей с одинаковыми лейблами

In [5]:
#Создаем датасет label_counts, в котором для каждого лейбла хранится число его повторений 
#(не берем лейблы "-", "S" и "Standard ")
label_counts = df.label.value_counts()[3:]

In [6]:
#Вычисление количества пар новостей, принадлежащих одной группе
sum(pd.Series.apply(label_counts, lambda x: x*(x-1)/2))

32980

In [ ]:
#Найдем отношение количества пар новостей, принадлежащих одной группе, в полному количеству пар новостей
float(sum(pd.Series.apply(label_counts, lambda x: x*(x-1)/2))) / (((df.shape[0]) * (df.shape[0] - 1))/2)
#Результат: 3.2039282132379393e-06
#Это еще одна причина, по которой обучиться будет невозможно: имеется ничтожно малое по сравнению с полным 
#количество пар новостей, принадлежащих одной группе. Классификатору будет выгодно говорить, что две новости
#не являются дубликатами, так как он будет ошибаться всего лишь в 3.2039282132379393e-06 случаях.

In [7]:
#Создание датасета, в котором хранятся индексы одинаковых лейблов
#Исключаем из датасета лейблы "-", "S", "Standard ", храним получившийся датасет в filtered_df
filtered_df = df[~df.label.isin(['-', 'S', 'Standard '])]
#Присваиваем label_col только колонку 'label'
label_col = filtered_df[['label']]
#Создаем колонку 'index', в которой содержатся старые индексы лейблов, для каждого лейбла индекс в датасете обновился
label_idx = label_col.reset_index(drop=False)
#Группируем по лейблам 
label_groupby = label_idx.groupby('label')
#Каждую запись колонки "index" превращаем в список
label_indices = label_groupby.apply(lambda x: list(x['index']))

#### Построение пар новостей, принадлежащих одной группе

In [8]:
#Применяем itertools.combinations из модуля itertools ко всем индексам лейблов из label_indices
import itertools
num_comb = pd.Series(label_indices.apply(lambda x: list(itertools.combinations(x, 2))))

In [9]:
#создаем три списка: в первом списке хранится индекс первой новости, во втором списке - второй новости, а в третьем 
#списке указано, совпадают ли лейблы этих индексов (так как изначально брались одинаковые лейблы, то для всех пар 
#индексов все значения списка same будут True)
index_0 = []
index_1 = []
same = []
#заполняем каждый список
for comb in num_comb:
    for index_comb in comb:
        index_0.append(index_comb[0])
        index_1.append(index_comb[1])
        same.append('True')

In [10]:
#Создаем список, в котором объединяем списки index_0, index_1, same.
df_index_pair = [[0] * 3 for m in range(len(index_0))]
for k in range(len(index_0)):
    df_index_pair[k][0] = index_0[k]
    df_index_pair[k][1] = index_1[k]
    df_index_pair[k][2] = same[k]

In [ ]:
#Преобразуем список в датафрейм 
df_index_pair = pd.DataFrame.from_records(df_index_pair, columns = ['index_0', 'index_1', 'same'])
df_index_pair

#### Построение пар новостей, не принадлежащих одной группе

In [12]:
#cоздаем два объекта, в каждом из которых содержится заданное количество n новостей
n = df_index_pair.shape[0]
label_0, label_1 = df.label[~df.label.isin(['-', 'S', 'Standard '])].sample(n), df.label[~df.label.isin(['-', 'S', 'Standard '])].sample(n)
#создаем два списка, в каждом из которых содержатся индексы выбранных новостей. Для этого для каждого значения k 
#в промежутке (0, df.shape[0]) проверяем, есть ли объект с таким индексом в созданных объектах. Если их нет и выходит ошибка, нужно
#продолжить цикл.
index_0 = []
index_1 = []
for k in range(df.shape[0]):
    try:
        label_0[k]
        index_0.append(k)
    except KeyError: 
        continue
for k in range(df.shape[0]):
    try: 
        label_1[k]
        index_1.append(k)
    except KeyError:
        continue
#создаем список, в котором содержатся значения 'True' или 'False'. Так как все значения должны быть 'False',
#то для каждой пары с 'True' делаем следующее: первую новость пары заменяем на случайно выбранную с помощью функции randint из модуля numpy. 
#cтрочка same.append('True') создана с целью убедиться в правильности построения датасета.
import numpy as np
same = []
for k in range(n):
    if df.label[index_0[k]] == df.label[index_1[k]]:
        while df.label[index_0[k]] == df.label[index_1[k]]:
            index_0[k] = np.random.randint(df.shape[0])
        if df.label[index_0[k]] == df.label[index_1[k]]:
            same.append('True')
        else:
            same.append('False')
    else:
        same.append('False')
#создаем список, в который включаем первый индекс, второй индекс, значения 'True' или 'False'
df_index_nonpair = [[0] * 3 for m in range(n)]
for k in range(n):
    df_index_nonpair[k][0] = index_0[k]
    df_index_nonpair[k][1] = index_1[k]
    df_index_nonpair[k][2] = same[k]
#преобразуем список в DataFrame
df_index_nonpair = pd.DataFrame.from_records(df_index_nonpair, columns = ['index_0', 'index_1', 'same'])

In [ ]:
#Объединяем датафреймы с одинаковыми лейблами и разными лейблами
result = pd.concat([df_index_pair, df_index_nonpair]).reset_index(drop=True)
result

## Обучение классификатора

#### Функция для обучения классификатора

In [35]:
#Создание функции для обучения классификатора. На вход поступают колонка со значениями True/False а также датафрейм, 
#содержащий колонки с признаками
def classifier(labels, df_selected):
    #преобразовываем значения True/False в массив со значениями 0/1
    labels = np.asarray(labels)
    #импортируем LabelEncoder
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    le.fit(labels)
    labels = le.transform(labels)
    #преобразовываем df_selected в словарь
    df_features = df_selected.to_dict(orient='records')
    #импортируем DictVectorizer, проводим векторизацию признаков с сохранением в features
    from sklearn.feature_extraction import DictVectorizer
    vec = DictVectorizer()
    features = vec.fit_transform(df_features).toarray()
    #разбиваем выборку на train/test в соотношении 80/20
    from sklearn.model_selection import train_test_split
    train, test, train_labels, test_labels = train_test_split(
      features, labels, 
      test_size=0.20, random_state=42)

    from sklearn.ensemble import RandomForestClassifier
    #инициализируем классификатор RandomForestClassifier как clf
    clf = RandomForestClassifier(random_state = 42)
    #обучаем классификатор
    clf.fit(train, train_labels)
    #сохраняем в acc_test точность предсказания
    acc_test = clf.score(test, test_labels)
    #возвращаем точность предсказания 
    #последняя строка показывает долю правильных ответов алгоритма.
    return "Test Accuracy:", acc_test


### Обучение классификатора без приведения слов к начальной форме

#### Токенизация текста без приведения слов к начальной форме

In [14]:
#Импортируем библиотеку nltk, а также стоп-слова
import nltk
from nltk.corpus import stopwords
#Создадим функцию токенизации текста без приведения слов к начальной форме
#принимаем на вход текст
def tokenize_me(file_text):
    #декодируем текст
    file_text = file_text.decode('utf-8')
    #Разбиваем текст на токены
    tokens = nltk.word_tokenize(file_text)
    #все буквы преобразуем в строчные
    tokens = [token.lower() for token in tokens]
    #импортируем библиотеку re, удаляем цифры из токенов
    import re
    tokens = [re.sub(r'\d', '', i) for i in tokens]
    #удаляем знаки препинания из токенов
    tokens = [i for i in tokens if i not in ('!', '', u'`', '.',':', '(', ')', u'№', u'–', u'«', u'»', ',', '...', '-', '[', ']', '{', '}', ';', "'", '"', '<', '>', '/', '?', '@', "#", '$', '%', '^', '&', '*', '_')]
    stop_words = stopwords.words('russian')
    #удаляем стоп-слова 
    tokens = [i for i in tokens if i not in stop_words]
    #удаляем оставшиеся знаки препинания
    tokens = [i.replace(u"`", "").replace(u"№", "").replace('-', "").replace(',', "").replace('.', "") for i in tokens]
    #удаляем токены, содержащие две или меньше букв
    tokens = [i for i in tokens if not len(i) <= 2]
    #возвращаем токены
    return tokens

#### Выделение признаков и обучение классификатора

##### Выделение признаков из пар текстов, которые состоят только из short-текстов новостей

In [16]:
#создаем списки, в которых будем хранить short-тексты для каждой новости из пары.
dfshort1 = []
dfshort2 = []
#к dfshort1 добавляем short-тексты первой новости пары (с result.index_0).
for k in result.index_0:
    dfshort1.append(df.short[k])
#к dfshort2 добавляем short-тексты второй новости пары (с result.index_1).
for k in result.index_1:
    dfshort2.append(df.short[k])
#создаем список dfshort_all, в котором будем хранить данные из списков dfshort1 и dfshort2. Добавляем два
#дополнительных столбца
dfshort_all = [[0] * 4 for i in range(len(dfshort1))]
for k in range(len(dfshort1)):
    dfshort_all[k][0] = dfshort1[k]
    dfshort_all[k][1] = dfshort2[k]
    dfshort_all[k][2] = 0
    dfshort_all[k][3] = 0
#Преобразовываем список dfshort_all в датафрейм с колонками 'index_0', 'index_1', 'same_items', 'same'
dfshort_all = pd.DataFrame.from_records(dfshort_all, columns = ['index_0', 'index_1', 'same_items', 'same'])
#Значениям колонки dfshort_all.same присваиваем значения колонки result.same
dfshort_all.same = result.same

In [18]:
#проводим токенизацию всех текстов из dfshort_all.index_0 и dfshort_all.index_1
pd.options.mode.chained_assignment = None
dfshort_all.index_0 = dfshort_all.index_0.apply(tokenize_me)
dfshort_all.index_1 = dfshort_all.index_1.apply(tokenize_me)
#помещаем токены каждого текста в множества
dfshort_all.index_0_sets = dfshort_all.index_0.apply(set)
dfshort_all.index_1_sets = dfshort_all.index_1.apply(set)
#находим число пересечений множеств, содержащих токены первой и второй новостей, присваиваем эти значения колонке same_items
for k in range(dfshort_all.shape[0]):
    dfshort_all.same_items[k] = len(dfshort_all.index_0_sets[k].intersection(dfshort_all.index_1_sets[k]))

In [19]:
#посчитаем, сколько признаков было найдено при пересечении множеств, содержащих токены short-текстов первой и второй новостей
sum(dfshort_all.same_items)

63389

In [40]:
#обучим классификатор, получим точность
classifier(dfshort_all.same, dfshort_all.drop(['index_0', 'index_1', 'same'], axis=1))

('Test Accuracy:', 0.88083687083080653)

##### Выделение признаков из пар текстов, которые состоят только из long-текстов новостей

In [21]:
#создаем списки, в которых будем хранить long-тексты для каждой новости из пары.
dflong1 = []
dflong2 = []
#к dflong1 добавляем long-тексты первой новости пары (с result.index_0).
for k in result.index_0:
    dflong1.append(df.long[k])
#к dflong2 добавляем long-тексты второй новости пары (с result.index_1).
for k in result.index_1:
    dflong2.append(df.long[k])
#создаем список dflong_all, в котором будем хранить данные из списков dflong1 и dflong2. Добавляем два
#дополнительных столбца
dflong_all = [[0] * 4 for i in range(len(dflong1))]
for k in range(len(dflong1)):
    dflong_all[k][0] = dflong1[k]
    dflong_all[k][1] = dflong2[k]
    dflong_all[k][2] = 0
    dflong_all[k][3] = 0
#Преобразовываем список dflong_all в датафрейм с колонками 'index_0', 'index_1', 'same_items', 'same'
dflong_all = pd.DataFrame.from_records(dflong_all, columns = ['index_0', 'index_1', 'same_items', 'same'])
#Значениям колонки dfshort_all.same присваиваем значения колонки result.same
dflong_all.same = result.same

In [22]:
#проводим токенизацию всех текстов из dflong_all.index_0 и dflong_all.index_1
pd.options.mode.chained_assignment = None
dflong_all.index_0 = dflong_all.index_0.apply(tokenize_me)
dflong_all.index_1 = dflong_all.index_1.apply(tokenize_me)
#помещаем токены каждого текста в множества
dflong_all.index_0_sets = dflong_all.index_0.apply(set)
dflong_all.index_1_sets = dflong_all.index_1.apply(set)
#находим число пересечений множеств, содержащих токены первой и второй новостей, присваеваем эти значения колонке same_items.
for k in range(dflong_all.shape[0]):
    dflong_all.same_items[k] = len(dflong_all.index_0_sets[k].intersection(dflong_all.index_1_sets[k]))

In [29]:
#посчитаем, сколько признаков было найдено при пересечении множеств, содержащих токены long-текстов первой и второй новостей
sum(dflong_all.same_items)

130817

In [36]:
#обучим классификатор, получим точность
classifier(dflong_all.same, dflong_all.drop(['index_0', 'index_1', 'same'], axis = 1))

('Test Accuracy:', 0.89084293511218926)

##### Выделение признаков из пар текстов, которые включают в себя short- и long-тексты новостей

In [103]:
#создаем списки, в которых будем хранить short- и long-текст для каждой новости из пары.
dfshort1 = []
dfshort2 = []
dflong1 = []
dflong2 = []
#к dfshort1 и dflong1 добавляем short- и long-тексты первой новости пары (с result.index_0).
for k in result.index_0:
    dfshort1.append(df.short[k])
    dflong1.append(df.long[k])
#к dfshort2 и dflong2 добавляем short- и long-тексты второй новости пары (с result.index_1).
for k in result.index_1:
    dfshort2.append(df.short[k])
    dflong2.append(df.long[k])
#создаем список dfshort_all, в котором будем хранить данные из списков dfshort1 и dfshort2. Добавляем два
#дополнительных столбца
dfshort_all = [[0] * 4 for i in range(len(dfshort1))]
for k in range(len(dfshort1)):
    dfshort_all[k][0] = dfshort1[k]
    dfshort_all[k][1] = dfshort2[k]
    dfshort_all[k][2] = 0
    dfshort_all[k][3] = 0
#создаем список dflong_all, в котором будем хранить данные из списков dflong1 и dflong2. Добавляем два
#дополнительных столбца
dflong_all = [[0] * 4 for i in range(len(dflong1))]
for k in range(len(dflong1)):
    dflong_all[k][0] = dflong1[k]
    dflong_all[k][1] = dflong2[k]
    dflong_all[k][2] = 0
    dflong_all[k][3] = 0
#Преобразовываем списки dfshort_all и dflong_all в датафреймы с колонками 'index_0', 'index_1', 'same_items', 'same'
dfshort_all = pd.DataFrame.from_records(dfshort_all, columns = ['index_0', 'index_1', 'same_items', 'same'])
dflong_all = pd.DataFrame.from_records(dflong_all, columns = ['index_0', 'index_1', 'same_items', 'same'])


In [104]:
#создаем датафреймы df_both1 и df_both2, в которых храним объединенные short- и long-тексты для каждой новости из пары
df_both1 = dfshort_all.index_0 + ' ' + dflong_all.index_0
df_both2 = dfshort_all.index_1 + ' ' + dflong_all.index_1
#создаем датафрем df_both с колонками 'index_0', 'index_1', 'same_items', 'same'. Колонке index_0 присваиваем df_both1,
#колонке index_1 присваиваем df_both2, колонке same присваиваем колонку result.same
df_both = pd.DataFrame(columns = ['index_0', 'index_1', 'same_items', 'same'])
df_both.index_0 = df_both1 
df_both.index_1 = df_both2
df_both.same_items = 0
df_both.same = result.same
pd.options.mode.chained_assignment = None
#проводим токенизацию всех текстов из index_0 и index_1
df_both.index_0 = df_both.index_0.apply(tokenize_me)
df_both.index_1 = df_both.index_1.apply(tokenize_me)
#помещаем токены каждого текста в множества
df_both.index_0_sets = df_both.index_0.apply(set)
df_both.index_1_sets = df_both.index_1.apply(set)
#находим число пересечений множеств, содержащих токены первой и второй новостей
for k in range(df_both.shape[0]):
    df_both.same_items[k] = len(df_both.index_0_sets[k].intersection(df_both.index_1_sets[k]))
#перемешиваем строки в df_both
#df_both = df_both.reindex(np.random.permutation(df_both.index))

In [105]:
#Обучаем классификатор, получаем точность (accuracy)
classifier(df_both.same, df_both.drop(['index_0', 'index_1', 'same'], axis=1))

0.95262280169799873

### Обучение классификатора с приведением слов к начальной форме

In [ ]:
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
mystem = Mystem(entire_input=False)
import re
#С учетом особенностей модуля mystem (который удаляет слова, содержащие цифры, из текста), создадим функцию, 
#которая будет предварительно убирать цифры, стоп-слова и слова,
#содержащие две и менее букв, а затем применим анализатор от mystem, который предоставит начальную форму слов и 
#грамматически охарактеризует их
def mystem_combined(file_text):
    #декодируем текст
    file_text = file_text.decode('utf-8')
    #токенизация текста при помощи nltk.word_tokenize (модуль nltk)
    tokens = nltk.word_tokenize(file_text)
    #все буквы в словах заменяем на строчные
    tokens = [token.lower() for token in tokens]
    #удаляем цифры
    tokens = [re.sub(r'\d', '', i) for i in tokens]
    #удаляем стоп-слова
    stop_words = stopwords.words('russian')
    tokens = [i for i in tokens if i not in stop_words]
    #удаляем слова, содержащие две или менее букв
    tokens = [i for i in tokens if not len(i) <= 2]
    #сливаем образовавшиеся токены в один текст
    file_text = ' '.join(tokens)
    #применяем анализатор mystem
    file_text = mystem.analyze(file_text)
    #теперь для каждого слова хранится грамматическая информация, начальная форма и исходный текст. Удалим исходный текст
    #за ненадобностью и в целях уменьшения объема памяти, занимаемого информацией о словах. Так как mystem применим 
    #только к русскоязычным словам, то для английских слов и нераспознанных русских слов грамматической информации не 
    #имеется. Поэтому при обращении к ней может возникнуть ошибка IndexError. Будем пропускать такие слова.
    for k in range(len(file_text)):
        try: 
            file_text[k][u'analysis'][0][u'gr']
            del file_text[k][u'text']
        except IndexError:
            pass
    #создаем новый список, добавляем к нему словари с информацией о словах. Если словари полностью совпадают, то лишний
    #словарь не добавляется в список
    new_file_text = []
    for x in file_text:
        if x not in new_file_text:
            new_file_text.append(x)
    #возвращаем список
    return new_file_text

### Обучение классификатора с выделением признаков: часть речи, "гео", "имя", "фам" и "отч"

In [ ]:
#имеется предварительно созданный файл, в котором каждой части речи присвоено какое-либо значение от 0 до 14, где
#цифра 0 обозначает, что часть речи не определена. Это может говорить о том, что слово не русскоязычное либо это слово
#не определено словарем (например, из-за ошибок в слове)
#импортируем файл characterisics.csv и дадим название его колонкам - word class и code
table_ch = pd.read_csv('characteristics.csv', sep=',', header = None)
table_ch.columns = ['word class', 'code']

In [ ]:
#создадим функцию, которая приведет всю информацию о слове к виду: word, x, y, где word - начальная форма слова,
#x - код части речи, y - цифра "0" - слово не является географическим названием, не является именем, фамилией или отчеством, 
#цифра "1" - слово является географическим названием, цирфа "2" - слово является именем, фамилией или отчеством
#импортируем модуль string (необходим для нахождения знаков препинания)
import string
#создадим предварительно вспомогательную функцию geo_name, которая будет часто использоваться в основной функции obrabotka
#на вход поступают слово m и список l
def geo_name(m, l):
    #если слово является географическим названием, к списку l добавить цифру "1"
    if m == u'гео':
        l.append(1)
    #если слово является именем, фамилией или отчеством, к списку l добавить цифру "2"
    elif (m == u"имя" or m == u"фам" or m == u"отч"):
        l.append(2)
    #если слово не имеет этих признаков, к списку l добавить цифру "0"
    else:
        l.append(0)
    #возвращаем l
    return l
#основная функция, на вход поступает столбец датафрейма
def obrabotka(series):
    #обращаемся к каждой строке столбца
    for p in range(series.shape[0]):
        #присваиваем строку line
        line = series[p]
        #обращаемся к каждому словарю строки, в котором содержится информация о слове
        for y in range(len(line)):
            #вводим переменные: 1) n - необходим для счета количества обработанных слов в строчке, в которой дается 
            #грамматическая характеристика слова; это число не должно превышать двух, когда n = 1, это значит,
            #что определена часть речи, когда n = 2, это значит, что определена часть речи и признак "гео", "имя" и т.п.
            #2) l - список, который и будет содержать новую информацию о слове; 3) m - слова, здесь происходит "сборка"
            #слова по буквам
            n = 0
            l = []
            m = ''
            #пробуем присоединить к списку l начальную форму слова
            try:
                l.append(line[y][u'analysis'][0][u'lex'])
                #проверяем, не было ли проверено уже два слова из раздела грамматики, если ответ утвердительный, то
                #исходная строка заменяется списком l.
                for k in range(len(line[y][u'analysis'][0][u'gr'])):
                    if n == 2:
                        pass
                    #в противном случае проверяем, является ли отдельно взятая буква из раздела грамматики знаком препинания
                    #(используется модуль string)
                    else:
                        if line[y][u'analysis'][0][u'gr'][k] in string.punctuation:
                            #если это так, то проверяется, является ли следующий символ знаком препинания. Если ответ 
                            #утвердительный, то ничего не делается (операция необходима для случаев, когда идут подряд
                            #два знака препинания. Если пропускать этот шаг, то второй слово из грамматического раздела
                            #может быть пропущено, а это значит, что возможно упускается признак)
                            #при этом может возникнуть ошибка - IndexError.
                            try:
                                if line[y][u'analysis'][0][u'gr'][k+1] in string.punctuation:
                                    pass
                                else:    
                                    #если следующий символ не является знаком препинания, то проводим следующие операции
                                    #если n = 0, то нужно выделить часть речи
                                    if n == 0:
                                        l.append(table_ch[table_ch['word class'] == m].index[0])
                                    #если n = 1, то нужно попробовать выделить географический или именной признак.
                                    #Для этого обращаемся к вспомогательной функции.
                                    if n == 1:
                                        l = geo_name(m, l)
                                    #Значение m обнуляем, чтобы затем наполнить его буквами следующего слова
                                    m = ''
                                    #значение n увеличиваем на единицу
                                    n += 1
                            except IndexError:
                                #если такая ошибка возникла, то нужно проверить значение n. Если оно равно нулю, то 
                                #к списку l добавляется код части речи слова. Так как больше информации о слове нет,
                                #то и географических и именных признаков нет, поэтому к списку также добавляется число "0".
                                if n == 0:
                                    l.append(table_ch[table_ch['word class'] == m].index[0])
                                    l.append(0)
                                #если значение n равно единице, то это значит, что часть речи была выделена, и нужно 
                                #проверить, является ли второе слово географическим или именным признаком. Для этого
                                #обращаемся к вспомогательной функции geo_name.
                                if n == 1:
                                    l = geo_name(m, l)
                                #больше информации о слове нет, поэтому приравниваем n к двум.
                                n == 2
                        else:
                            #если следующий символ не явлется знаком препинания, то "собирание" слова еще не завершено.
                            #Поэтому к m присоединяем символ.
                            m += line[y][u'analysis'][0][u'gr'][k]
                            try:
                                #проверяем, существует ли следующий символ
                                line[y][u'analysis'][0][u'gr'][k+1]
                            except IndexError:
                                #если его не существует, то это значит, что грамматическая информация о слове закончилась.
                                #Так как в ней содержится по крайней мере одно слово, то n в данном случае не может быть
                                #равен нулю. Значит, часть речи уже выделена, остается попытаться выделить географический
                                #или именной признак. Обращаемся к вспомогательной функции geo_name.
                                if n == 1:
                                    l = geo_name(m, l)
                                    n = 2
                #После всех операций над символами заменяем исходную строку на список l.
                line[y] = l
            #если при попытке присоединения к списку l начальной формы слова возникает ошибка IndexError, то
            #в список l добавляется исходная форма слова, считается, что никаких признаков слово не имеет
            except IndexError:
                l.append(line[y][u'text'])
                l.append(0)
                l.append(0)
            #исходную информацию о слове заменяем информацией из списка l
                line[y] = l

In [ ]:
#После выделения именных и географических признаков переходим к выделению количественных признаков. 
#импортируем модуль numpy
import numpy as np
#создаем columns, которому присваиваем значения от 0 до 14
columns = [np.arange(15)]
#создаем три датафрема с колонками из columns, количество строк определено количеством строк датафрема result
df_class1 = pd.DataFrame(columns = columns, index = np.arange(result.shape[0]))
df_class2 = pd.DataFrame(columns = columns, index = np.arange(result.shape[0]))
df_class3 = pd.DataFrame(columns = columns, index = np.arange(result.shape[0]))
#создаем функцию, при выполнении которой получим датафрейм с количеством совпадающих слов двух новостей
#по той или иной части речи. На вход поступает колонка датафрейма.
def X2_func(x, y):
    for k in range(x, y):
        p = result.index_0[k]
        for m in range(len(dfboth.both[p])):
            try:
                df_class1[dfboth.both[p][m][1]][k].add(dfboth.both[p][m][0])          
            except AttributeError:
                df_class1[dfboth.both[p][m][1]][k] = set()
                df_class1[dfboth.both[p][m][1]][k].add(dfboth.both[p][m][0])
        p1 = result.index_1[k]
        for m in range(len(dfboth.both[p1])):
            try:
                df_class2[dfboth.both[p1][m][1]][k].add(dfboth.both[p1][m][0])            
            except AttributeError:
                df_class2[dfboth.both[p1][m][1]][k] = set()
                df_class2[dfboth.both[p1][m][1]][k].add(dfboth.both[p1][m][0])
        for l in range(15):
            try:
                df_class3[l][k] = len(df_table1[l][k].intersection(df_table2[l][k]))
            except AttributeError:
                df_class3[l][k] = 0
            except TypeError:
                df_class3[l][k] = 0

In [ ]:
columns = [np.arange(3)]
df_table1 = pd.DataFrame(columns = columns, index = np.arange(result.shape[0]))
df_table2 = pd.DataFrame(columns = columns, index = np.arange(result.shape[0]))
df_table3 = pd.DataFrame(columns = columns, index = np.arange(result.shape[0]))
def X3_func(x, y):
    for k in range(x, y):
        p = result.index_0[k]
        for m in range(len(dfboth.both[p])):
            try:
                df_table1[dfboth.both[p][m][2]][k].add(dfboth.both[p][m][0])          
            except AttributeError:
                df_table1[dfboth.both[p][m][2]][k] = set()
                df_table1[dfboth.both[p][m][2]][k].add(dfboth.both[p][m][0])
        p1 = result.index_1[k]
        for m in range(len(dfboth.both[p1])):
            try:
                df_table2[dfboth.both[p1][m][2]][k].add(dfboth.both[p1][m][0])            
            except AttributeError:
                df_table2[dfboth.both[p1][m][2]][k] = set()
                df_table2[dfboth.both[p1][m][2]][k].add(dfboth.both[p1][m][0])
        for l in range(3):
            try:
                df_table3[l][k] = len(df_table1[l][k].intersection(df_table2[l][k]))
            except AttributeError:
                df_table3[l][k] = 0
            except TypeError:
                df_table3[l][k] = 0

In [ ]:
geo_class = pd.concat([df_class3, df_table3], axis=1)
geo_class['same'] = result.same
get_class = geo_class.reindex(np.random.permutation(geo_class.index))

In [ ]:
df_class3['same'] = result.same
df_table3['same'] = result.same
df_class3 = df_class3.reindex(np.random.permutation(df_class3.index))
df_table3 = df_table3.reindex(np.random.permutation(df_table3.index))

In [ ]:
classifier(df_class3.same, df_class3.drop(['same'], axis=1))

In [ ]:
classifier(df_table3.same, df_table3.drop(['same'], axis=1))

In [ ]:
classifier(geo_class.same, geo_class.drop(['same'], axis=1))